In [2]:
from dotenv import load_dotenv
import os
from openai import OpenAI
# .env 파일에서 환경 변수를 불러옵니다.
load_dotenv()

# 환경 변수 읽기
openai_api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 라이브러리가 환경 변수에서 키를 읽을 수 있도록 설정
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

print("API Key configured:", "OPENAI_API_KEY" in os.environ)

API Key configured: True


In [3]:
client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o-mini", #omni  model
  messages=[
      {'role':'user', 'content':'파이썬에 대해 알려줘'}
  ]
  )


In [7]:
message={'role':'user', 'content':'오늘 점심메뉴 추천 좀 '}
response = client.chat.completions.create(
  model="gpt-4o-mini", #omni  model
  messages=[
      message
  ]
  )
assistant_reply = response.choices[0].message.content
print('ChatGPT 답변:', assistant_reply)

ChatGPT 답변: 오늘 점심으로 어떤 메뉴를 드시는 게 좋을까요? 몇 가지 추천해드릴게요!

1. **비빔밥** - 다양한 채소와 고기를 곁들인 비빔밥은 맛도 좋고 영양도 가득합니다.
2. **김치찌개** - 따뜻한 김치찌개에 밥 한 공기를 더하면 든든한 점심이 될 거예요.
3. **치킨 샐러드** - 건강하게 먹고 싶다면 구운 치킨과 신선한 채소로 구성된 샐러드도 좋은 선택입니다.
4. **떡볶이와 튀김** - 매콤한 떡볶이와 바삭한 튀김은 간편하면서도 맛있는 점심이 됩니다.
5. **파스타** - 크림소스나 토마토소스 파스타를 즐기면 간단하지만 맛있게 해결할 수 있어요.

어떤 메뉴가 마음에 드시나요? 추가적인 조정이나 다른 종류의 추천이 필요하시면 말씀해 주세요!


# 출력 형식 지정

In [9]:
message={'role':'user', 'content':'오늘 서울날씨 json형태로 제공해 키는 weather,temperature 로 설정해'}
response = client.chat.completions.create(
  model="gpt-4o-mini", #omni  model
  messages=[
      message
  ],
  response_format = {'type':'json_object'}
  )
assistant_reply = response.choices[0].message.content
print('ChatGPT 답변:', assistant_reply)

ChatGPT 답변: {
  "weather": "맑음",
  "temperature": "15°C"
}


In [5]:
message={'role':'user', 'content':'오늘 서울, 부산날씨 json형태로 제공해 키는 weather,temperature 로 설정해 날씨는 기상청을 참고해서꺼내'}
response = client.chat.completions.create(
  model="gpt-4o-mini", #omni  model
  messages=[
      message
  ],
  response_format = {'type':'json_object'}
  )
assistant_reply = response.choices[0].message.content
print('ChatGPT 답변:', assistant_reply)

ChatGPT 답변: {
  "weather": {
    "seoul": {
      "weather": "맑음",
      "temperature": "20°C"
    },
    "busan": {
      "weather": "구름 많음",
      "temperature": "22°C"
    }
  }
}

  
 

 


In [7]:
import json
data = json.loads(assistant_reply)
data.keys()

dict_keys(['weather'])

In [8]:
# 도시 x의 날씨는 현재 .. 이고, 온도는 .. 입니다
message={'role':'user', 'content':'오늘 Seoul의 날씨를 "도시 의 날씨는 현재 .. 이고, 온도는 .. 입니다"의 형태로 반환'}
response = client.chat.completions.create(
  model="gpt-4o-mini", #omni  model
  messages=[
      message
  ]
  )
assistant_reply = response.choices[0].message.content
print('ChatGPT 답변:', assistant_reply)

ChatGPT 답변: 오늘 서울의 날씨는 현재 맑고, 온도는 약 20도입니다. (참고로 실제 날씨 정보는 확인해 주셔야 합니다.)


In [ ]:
messages = {"role": "user", "content": "(뉴욕, 보스턴, 토론토)의 날씨를 json형태로 제공해 키는 weather,temperature 로 설정"}
res = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[messages],
    response_format={"type": "json_object"}
)
reply = res.choices[0].message.content
print(reply)
mission = json.loads(reply)

{"weather": {
    "New York": {
        "temperature": "18°C",
        "description": "Sunny"
    },
    "Boston": {
        "temperature": "16°C",
        "description": "Partly cloudy"
    },
    "Toronto": {
        "temperature": "15°C",
        "description": "Rain"
    }
}}
weather 의 날씨는 None 이고 온도는 None 입니다.


In [24]:
for city, info in mission['weather'].items():
    print(f"{city} 의 날씨는 {info.get('description')} 이고 온도는 {info.get('temperature')} 입니다.")

New York 의 날씨는 Sunny 이고 온도는 18°C 입니다.
Boston 의 날씨는 Partly cloudy 이고 온도는 16°C 입니다.
Toronto 의 날씨는 Rain 이고 온도는 15°C 입니다.


# CoT 프롬프트

In [19]:
question = '사과 10개 가지고 있었는데 그 중 3개를 먹은 뒤에 5개를 더 구입했어 그럼 사과는 몇개가 남았어?'
res = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':question}
  ],
    temperature=0
)
print('cot 적용전: ', res.choices[0].message.content)

cot 적용전:  처음에 사과가 10개 있었고, 3개를 먹었으니 남은 사과는 10 - 3 = 7개입니다. 그 후에 5개를 더 구입했으니, 7 + 5 = 12개가 됩니다. 따라서 현재 사과는 12개가 남았습니다.


In [20]:
question = '사과 10개 가지고 있었는데 그 중 3개를 먹은 뒤에 5개를 더 구입했어 그럼 사과는 몇개가 남았어?'
cot_prompt= '(생각을 단계별로 진행해)\n' + question
res = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':cot_prompt}
  ],
    temperature=0
)
print('cot 적용전: ', res.choices[0].message.content)

cot 적용전:  단계별로 생각해보겠습니다.

1. 처음에 사과가 10개 있었습니다.
2. 그 중 3개를 먹었습니다. 
   - 10개 - 3개 = 7개
3. 이후에 5개를 더 구입했습니다.
   - 7개 + 5개 = 12개

결론적으로, 사과는 12개가 남았습니다.


In [22]:
question = '어떤 수를 3배한 결과에 4를 더하면 19이 됩니다. 그 수는 무엇일까요?'
res = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':question}
  ],
    temperature=0
)
print('cot 적용전: ', res.choices[0].message.content)

cot 적용전:  어떤 수를 \( x \)라고 할 때, 문제에서 주어진 조건을 식으로 표현하면 다음과 같습니다.

\[
3x + 4 = 19
\]

이제 이 식을 풀어보겠습니다.

1. 양변에서 4를 빼줍니다.

\[
3x = 19 - 4
\]
\[
3x = 15
\]

2. 양변을 3으로 나눕니다.

\[
x = \frac{15}{3}
\]
\[
x = 5
\]

따라서, 그 수는 \( 5 \)입니다.


# Self-Consistency 디코딩

In [29]:
question = '어떤 수를 3배한 결과에 4를 더하면 19이 됩니다. 그 수는 무엇일까요?'
message=[{'role':'user', 'content': question}]
answers = []
   
for i in range(5):
    res = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=message,
    temperature=1.0
    )
    answer = res.choices[0].message.content.strip()
    answers.append(answer)
    print(f' 응답 {i+1} : {answer}')

 응답 1 : 어떤 수를 \( x \)라고 할 때, 문제의 조건을 식으로 표현할 수 있습니다:

\[
3x + 4 = 19
\]

이제 이 식을 풀어보겠습니다. 먼저, 양변에서 4를 빼줍니다:

\[
3x = 19 - 4
\]
\[
3x = 15
\]

그 다음, 양쪽을 3으로 나누어 줍니다:

\[
x = \frac{15}{3}
\]
\[
x = 5
\]

따라서, 그 수는 \( 5 \)입니다.
 응답 2 : 어떤 수를 \( x \)라고 했을 때, 문제에서 주어진 조건을 방정식으로 표현할 수 있습니다. 

3배한 결과에 4를 더하면 19가 된다는 것은 다음과 같은 방정식으로 나타낼 수 있습니다:

\[ 3x + 4 = 19 \]

이 방정식을 풀기 위해, 먼저 양쪽에서 4를 빼줍니다:

\[ 3x = 19 - 4 \]

\[ 3x = 15 \]

그 다음, 양쪽을 3으로 나누어 \( x \) 값을 구합니다:

\[ x = \frac{15}{3} \]

\[ x = 5 \]

따라서, 그 수는 \( 5 \)입니다.
 응답 3 : 어떤 수를 \( x \)라고 가정합시다. 주어진 조건을 식으로 표현하면 다음과 같습니다.

\[
3x + 4 = 19
\]

이제 이 식을 풀어보겠습니다. 먼저 양쪽에서 4를 빼줍니다.

\[
3x = 19 - 4
\]

\[
3x = 15
\]

그 다음 양쪽을 3으로 나누면,

\[
x = \frac{15}{3}
\]

\[
x = 5
\]

따라서 그 수는 \( 5 \)입니다.
 응답 4 : 문제를 수식으로 표현해보겠습니다. 어떤 수를 \( x \)라 할 때, 주어진 조건을 수식으로 나타내면 다음과 같습니다:

\[ 3x + 4 = 19 \]

이 식을 풀어보겠습니다.

1. 양변에서 4를 빼줍니다:

\[ 3x = 19 - 4 \]
\[ 3x = 15 \]

2. 양변을 3으로 나누어 줍니다:

\[ x = \frac{15}{3} \]
\[ x = 5 \]

따라서, 그 수는 **5**입니다.
 응답 5 : 문제를 수식으로 표현

In [31]:
final_answer = [ans.split()[-1] for ans in answers]
import collections
counter = collections.Counter(final_answer)
fans = counter.most_common(1)[0][0]
print(f'최종답: {fans}')

최종답: \)입니다.


# Reflexion (자기 피드백 반복 프롬프트 설계)

In [33]:
question = '7의 2승에 5을 곱한 값'
res = client.chat.completions.create(
    model="gpt-4o-mini",
  messages=[
      {'role':'user', 'content':question}
  ],
    temperature=0
)
answer = res.choices[0].message.content.strip()
correct_answer = 245
if str(correct_answer) in answer:
    feedback = '정답입니다'
    need_retry = False
    
else:
    feedback = '틀렸어요. 힌트는 7의 제곱값을 정확히 구한 다음 곱하세요'
    need_retry = True
print(feedback)

정답입니다


In [38]:
question = '7의 2승에 5을 곱한 값'
res = client.chat.completions.create(
    model="gpt-3.5-turbo",
  messages=[
      {'role':'user', 'content':question}
  ],
    temperature=0
)
answer = res.choices[0].message.content.strip()
correct_answer = 245
if str(correct_answer) in answer:
    feedback = '정답입니다'
    need_retry = False
    
else:
    feedback = '틀렸어요. 힌트는 7의 제곱값을 정확히 구한 다음 곱하세요'
    need_retry = True
print(feedback)
print(answer)

틀렸어요. 힌트는 7의 제곱값을 정확히 구한 다음 곱하세요
49


In [ ]:
if need_retry:
    retry_prompt = f'이전 답변 : {answer}, 피드백 : {feedback} \n 다시 답을 구하세요'
    res2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {'role':'user', 'content':question}
    ],
    temperature=1
    )
    answer2 = res2.choices[0].message.content.strip()
    print(answer2)

7의 2승은 49이고, 여기에 5를 곱하면 245가 됩니다.


In [42]:
prompt = [{'role':'user', 'content':'기후변화의 원인과 해결방안에 대한 간략한 에세이를 작성하세요'}]
res3 = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompt,
    temperature=1.0 
)
answer3 = res3.choices[0].message.content.strip()
print(answer3)

### 기후변화의 원인과 해결방안

기후변화는 현대 사회가 직면한 가장 심각한 환경문제 중 하나로, 지구의 평균 기온 상승과 이로 인해 발생하는 다양한 이상 기후 현상을 포함합니다. 기후변화의 주요 원인은 인간 활동에 의해 발생한 온실가스의 배출입니다. 산업화, 도시화, 그리고 화석연료의 사용 증가는 이산화탄소(CO2), 메탄(CH4) 등 온실가스를 대량으로 방출하게 되었고, 이는 지구의 온도를 상승시키는 주된 요인으로 작용하고 있습니다.

기후변화의 결과로는 해수면 상승, 극단적인 기후현상(폭염, 폭우, 가뭄 등) 및 생태계 변화 등이 있습니다. 이러한 변화는 자연환경뿐만 아니라 인류의 생활에도 심각한 영향을 미치고 있습니다. 농업 생산성이 저하되고, 식수자원 부족이 심화되며, 질병의 확산 등이 그 예입니다.

기후변화를 해결하기 위해서는 다각적인 접근이 필요합니다. 첫째, 에너지 효율을 높이고 재생 가능 에너지원(태양광, 풍력 등)을 적극적으로 활용해야 합니다. 둘째, 교통수단의 전환과 대중교통 시스템 개선을 통해 화석연료 사용을 줄여야 합니다. 셋째, 나무 심기 및 산림 보호를 통한 탄소 흡수 능력을 증가시켜야 합니다. 넷째, 국가 간 협력과 정책적 노력이 필요합니다. 국제적인 기후 협약, 예를 들어 파리협정과 같은 기후 변화 대응에 대한 공동의 약속을 통해 세계 각국이 한 방향으로 나아가야 합니다.

결론적으로, 기후변화는 인류의 생존에 직접적인 위협을 주는 문제로, 이를 해결하기 위한 노력이 절실합니다. 개인의 작은 행동에서부터 정부 및 국제 사회의 정책적 대응까지, 모든 주체가 협력해 지속 가능한 미래를 만들어가야 할 것입니다.


In [44]:
correct_answer = '전력소비'
if correct_answer in answer3:
    feedback = '정답입니다'
    need_retry = False
    
else:
    feedback = '전력 소비 내용을 포함해서 서술하시오'
    need_retry = True

if need_retry:
    retry_prompt = f'이전 답변 : {answer3}, 피드백 : {feedback} \n 다시 답을 구하세요'
    res4 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=prompt,
    temperature=1
    )
    answer4 = res4.choices[0].message.content.strip()
    print(answer4)

**기후변화의 원인과 해결방안**

기후변화는 현대 인류가 직면한 가장 심각한 환경적 도전 중 하나로, 지구의 평균 기온 상승, 해수면 상승, 극단적인 날씨 현상 등 다양한 형태로 나타납니다. 이러한 기후변화의 주요 원인은 온실가스의 배출 증가로, 이는 주로 화석 연료의 연소, 산업 활동, 농업, 그리고 임업 등 인간 활동에서 기인합니다. 특히 이산화탄소(CO2), 메탄(CH4), 그리고 아산화질소(N2O)와 같은 온실가스는 대기 중에 축적되어 지구의 열을 가두는 역할을 합니다.

기후변화의 해결을 위해서는 여러 가지 접근이 필요합니다. 첫 번째로, 에너지 전환이 중요합니다. 화석 연료에서 재생 가능 에너지로의 전환을 통해 온실가스 배출을 줄일 수 있습니다. 태양광, 풍력, 수력 등 청정 에너지원의 활용을 확대해야 합니다.

두 번째로, 효율적인 에너지 사용이 필요합니다. 건축물, 교통수단, 산업 분야에서 에너지 효율성을 높이는 기술과 정책을 도입하여 에너지 소비를 줄일 수 있습니다. 예를 들어, 에너지 효율이 높은 가전제품을 사용하고 대중교통을 이용하는 습관을 기르는 것이 좋습니다.

세 번째로, 자연 기반 해결책(Nature-based Solutions)을 통해 탄소를 흡수하는 생태계를 보호하고 복원하는 것이 필요합니다. 산림을 보호하고, 도시 녹지를 확장하며, 농업에서 지속 가능한 방법을 채택하여 자연의 탄소 흡수 능력을 높여야 합니다.

마지막으로, 국제적인 협력이 필수적입니다. 각국 정부, 기업, 그리고 시민들이 협력하여 기후 변화 대응을 위한 정책을 수립하고 실천해야 합니다. 파리협약과 같은 국제적 합의는 이러한 협력의 밑바탕이 됩니다.

결론적으로, 기후변화는 인류가 함께 해결해야 할 문제입니다. 지속 가능한 발전과 환경 보호를 위한 실천을 통해 우리는 지구를 지키고 후세에 아름다운 자연을 물려줄 수 있습니다. 기후변화는 단순한 환경 문제를 넘어 우리 모든 삶과 미래에 걸쳐 있는 요소임을 염두에 두고 행동해야 할 때입니다.
